# Nettoyage des données

## Import des données

On a récupéré des textes de questions du site stackexchange, en préselectionnant les questions ayant au moins un certain nombre de vues, de réponses, d'avis positifs et surtout de tags associés à la question.

Commençons par importer les bibliothèques requises pour le nettoyage et traitement des données.

In [1]:
#import des bibliothèques requises
import pandas as pd
import numpy as np
#Gestion des strings
import string
import re
#Traitement de textes
import nltk
from nltk.corpus import stopwords
import spacy

On charge désormais la base de données dans un dataframe avant de la visualiser.

In [2]:
#On importe le jeu de données
data = pd.read_csv('QueryResults.csv')
#On affiche sa taille et les noms des colonnes
print(data.shape)
print(data.columns)
#On affiche les premières lignes pour mieux comprendre les données
data.head()

(27128, 8)
Index(['Title', 'Body', 'Tags', 'Id', 'Score', 'ViewCount', 'FavoriteCount',
       'AnswerCount'],
      dtype='object')


,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount
0,Mercator longitude and latitude calculations t...,<p>I have this image. It's a map of the UK (no...,<javascript><geolocation><projection><processi...,2103924,19,30199,29,10
1,Starting point for learning CAD/CAE file formats?,<p>We are developing some stress and strain an...,<c++><file-format><autocad><cad><solid-bodies>,1024323,6,6097,11,6
2,LINQ query on a DataTable,<p>I'm trying to perform a LINQ query on a Dat...,<c#><.net><linq><datatable><.net-3.5>,10855,1107,1042939,215,22
3,How to overload std::swap(),<p><code>std::swap()</code> is used by many st...,<c++><performance><optimization><stl><c++-faq>,11562,122,34718,50,4
4,How do you get a directory listing in C?,<p>How do you scan a directory for folders and...,<c><file><directory><cross-platform><common-ta...,12489,65,136135,21,9


On commence par se défausser des variables qui nous seront inutiles ; on ne gardera que le titre et le corps de la question (qui constituent le texte à étudier) ainsi que les tags pour l'apprentissage supervisé.

In [3]:
data.drop(columns = ['Id', 'Score', 'ViewCount', 'FavoriteCount', 'AnswerCount'], inplace = True)
data.head()

,Title,Body,Tags
0,Mercator longitude and latitude calculations t...,<p>I have this image. It's a map of the UK (no...,<javascript><geolocation><projection><processi...
1,Starting point for learning CAD/CAE file formats?,<p>We are developing some stress and strain an...,<c++><file-format><autocad><cad><solid-bodies>
2,LINQ query on a DataTable,<p>I'm trying to perform a LINQ query on a Dat...,<c#><.net><linq><datatable><.net-3.5>
3,How to overload std::swap(),<p><code>std::swap()</code> is used by many st...,<c++><performance><optimization><stl><c++-faq>
4,How do you get a directory listing in C?,<p>How do you scan a directory for folders and...,<c><file><directory><cross-platform><common-ta...


## Formatage des textes

On va afficher les textes de la première entrée pour voir sous quel(s) format(s) ils sont rédigés. 

In [4]:
print("Titre :")
print(data.iloc[0,0], '\n')
print("Corps du texte :")
print(data.iloc[0,1], '\n')
print("Tags associés :")
print(data.iloc[0,2])

Titre :
Mercator longitude and latitude calculations to x and y on a cropped map (of the UK) 

Corps du texte :
<p>I have this image. It's a map of the UK (not including Southern Ireland):</p>

<p><a href="https://i.stack.imgur.com/KUcpE.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/KUcpE.png" alt="UK Map"></a> </p>

<p>I have successfully managed to get a latitude and longitude and plot it onto this map by taking the leftmost longitude and rightmost longitude of the UK and using them to work out where to put the point on the map.</p>

<p>This is the code (for use in Processing.js but could be used as js or anything):</p>

<pre><code>// Size of the map
int width = 538;
int height = 811;
// X and Y boundaries
float westLong = -8.166667;
float eastLong = 1.762833;
float northLat = 58.666667;
float southLat = 49.95;

void drawPoint(float latitude, float longitude){

 fill(#000000);

 x = width * ((westLong-longitude)/(westLong-eastLong));
 y = (height * ((northLat-lat

Le corps du texte est au format HTML, avec des **balises** permettant de structurer le texte. Il nous faut comprendre cette mise en forme pour la transformer en quelque chose de plus manipulable. Heureusement pour nous toutes ces balises se trouvent entre chevrons (par exemple "**<p\>**").

In [5]:
#On va chercher les balises utilisées dans notre corpus en cherchant les chevrons dans le texte
#(le paramètre "interdites" permet de lister des balises qu'on ne souhaite pas détecter)
def find_balises(textes, interdites):
    balises = []
    for t in textes :
        #On cherche une première balise
        temp = t.find('<')
        n = len(t)
        while temp >= 0:
            #Certaines balises prennent des paramètres entre les chevrons, il faut donc s'arrêter
            #au chevron fermant OU au premier espace
            end1 = t.find('>', temp+1, n)
            end2 = t.find(' ', temp+1, n)
            #On identifie la balise que l'on vient de trouver
            candidat = t[temp+1:min_modif(end1, end2, n)]
            #Si cette balise est nouvelle (et n'est pas "interdite")...
            if (candidat not in balises) and (candidat not in interdites):
                #... on l'ajoute à la liste
                balises.append(candidat)
            #On cherche la balise suivante (s'il n'y en a pas la fonction renvoie -1)
            temp = t.find('<', temp+1, n)
    return balises

#La fonction "find" pouvant renvoyer -1 il faut adapter la fonction "min" à ces cas particuliers
#en faisant comme si ce -1 valait plus que n, la longueur maximale possible dans ce contexte
def min_modif(a,b,n):
    if a < 0:
        if b < 0:
            return n
        else:
            return b
    elif b < 0:
        return a
    else:
        return min(a,b)

In [6]:
#On affiche les balises détectées dans le corps du premier texte (la liste interdite est vide pour l'instant)
balises_debut = find_balises([data.iloc[0,1]], [])
print(balises_debut)

['p', '/p', 'a', 'img', '/a', 'pre', 'code', '/code', '/pre']


Notre fonction pour détecter les balises de traitement de texte semble marcher, on va pouvoir lister les balises présentes dans notre corpus et les traiter convenablement. Celles déjà trouvées sont **<p\>** qui indique un paragraphe, **<a ...\>** qui permet d'insérer un lien hypertexte (qui est détaillé entre les chevrons), **<img\>** qui permet d'insérer une image, **<pre\>** qui permet d'insérer du texte formaté et **<code\>** qui permet d'écrire des lignes de code.

In [7]:
#On va chercher les balises des corps des textes du corpus, en excluant celles déjà trouvées dans le premier texte
balises_total = find_balises(data['Body'], balises_debut)
print(balises_total)
#On en profite pour lister les balises présentes dans les titres des questions (même si le premier n'en contenait pas)
balises_titres = find_balises(data['Title'], balises_debut)
print(balises_titres)

['strong', '/strong', 'em', '/em', 'ul', 'li', '/li', '/ul', 'blockquote', '/blockquote', 'hr', 'ol', '/ol', 'br', 'b', '/b', 'div', '/div', 'h2', '/h2', 'h3', '/h3', 'strike', '/strike', 'sup', '/sup', 'sub', '/sub', 'br/', 'A', '/A', 'hr/', 'h1', '/h1', 'kbd', '/kbd', 'h4', '/h4', 'hR', 'i', '/i', 's', '/s', 'del', '/del', 'table', 'thead', 'tr', 'th', '/th', '/tr', '/thead', 'tbody', 'td', '/td', '/tbody', '/table', 'BR', 'UL', 'LI', '/UL', 'HR', 'bR', 'a\nhref="http://java.dzone.com/articles/spring-test-mvc-junit-testing"', 'a\nhref="http://techdive.in/solutions/how-mock-securitycontextholder-perfrom-junit-tests-spring-controller"', 'a\nhref="https://stackoverflow.com/questions/5403818/how-to-junit-tests-a-preauthorize-annotation-and-its-spring-el-specified-by-a-s"']
['T', '', '<', 'List<int', 'int', 'XXX', 'deny', 'TKey,', 'meta', 'value', 'DerivedClass', 'BaseClass', 'type', '<<<<<<', '<<<<<', '<<<<', '<<<', '<<', 'string', 'object', '?', 'Animal', 'input', 'video', 'String', 'Fu

Beaucoup de nouvelles balises apparaissent, et certaines échappent à notre connaissance du html. Nous allons donc chercher des exemples de textes utilisant ces balises inconnues pour voir s'il faut conserver leur contenu ou non.

In [8]:
#La fonction va chercher dans nos textes un exemple pour chaque balise donnée en paramètre
def chercher_exemples(textes, balises):
    result = []
    for t in textes :
        #On commence par chercher une balise
        temp = t.find('<')
        n = len(t)
        while temp >= 0:
            end1 = t.find('>', temp+1, n)
            end2 = t.find(' ', temp+1, n)
            candidat = t[temp+1:min_modif(end1, end2, n)]
            #Si la balise est listée en paramètre...
            if (candidat in balises):
                #... on cherche la balise fermante associée
                temp_close = t.find('</'+candidat, end1+1, n)
                temp_end = t.find('>', temp_close+1, n)
                #On sauvegarde le contenu entre les deux balises
                result.append(t[temp:temp_end+1])
                #On enlève cette balise de la liste des balises recherchées
                balises.remove(candidat)
            temp = t.find('<', temp+1, n)
    return result

In [9]:
#On liste les balises qui nous posent problème
balises_inconnues = ['b', 'div', 'strike', 'kbd', 'i', 's', 'del']
#On cherche des exemples pour chacune de ces balises
exemples = chercher_exemples(data['Body'], balises_inconnues)
#Et on les affiche
for t in exemples:
    print(t)

<b>Prototype</b>
<div class="snippet" data-lang="js" data-hide="false" data-console="false" data-babel="false">
<div class="snippet-code">
<pre class="snippet-code-js lang-js prettyprint-override"><code>function myJsFunc() {
    alert("myJsFunc");
}</code></pre>
<pre class="snippet-code-html lang-html prettyprint-override"><code>&lt;a href="#" onclick="myJsFunc();"&gt;Run JavaScript Code&lt;/a&gt;</code></pre>
</div>
<strike><a href="http://www.bestechvideos.com/2008/06/08/dimecasts-net-introduction-to-mocking-with-moq" rel="noreferrer">http://www.bestechvideos.com/2008/06/08/dimecasts-net-introduction-to-mocking-with-moq</a></strike>
<kbd>"<strong>LATENCY</strong>"</kbd>
<i>(*) I edited my question based in the <a href="https://stackoverflow.com/questions/4046/can-someone-give-me-a-working-example-of-a-buildxml-for-an-ear-that-deploys-in-#4298">suggestion</a> of to use pastebin.ca</i>
<s>Unboxed (immutable)</s>
<del>.NET Framework</del>


Nous allons maintenant pouvoir mettre nos textes au format qui nous intéresse : un *string* en anglais, sans balises de formatage html. Il nous faudra pour cela distinguer deux types de balises : d'un côté celles qui mettent en forme le texte qui nous intéresse (par exemple en le mettant en gras ou en indiquant les paragraphes), de l'autre celles qui permettent d'insérer du contenu non textuelle (liens hypertextes, images, extraits de codes...). Dans le premier cas il nous faudra simplement supprimer les balises (i.e. ce qui se trouve entre "**\<**" et "**\>**"), tandis que dans le second cas il faudra également supprimer ce qui se trouve entre la balise ouvrante **<balise\>** et la balise fermante associée **<\balise\>**.

In [10]:
#On va formater nos textes en ôtant les balises. Pour certaines d'entre elles (listées dans "bal_suppr")
#il faudra aussi supprimer le contenu ENTRE les balises (par exemple les bouts de code ou les insertions d'images)
def formatage_texte(texte, bal_suppr):
    #On cherche une première balise
    temp = texte.find('<')
    n = len(texte)
    while temp >= 0:
        #On identifie la balise en cherchant le premier espace ou chevron fermant
        end1 = texte.find('>', temp+1, n)
        end2 = texte.find(' ', temp+1, n)
        candidat = texte[temp+1:min_modif(end1, end2, n)]
        #On initialise le point où redémarrer
        restart = temp+1
        #On vérifie qu'il s'agisse bien d'une balise
        if end1 > 0:
            #Si la balise est dans "balise_suppr"...
            if (candidat in bal_suppr):
                #... on cherche la balise fermante associée
                temp_close = texte.find('</'+candidat, end1+1, n)
                temp_end = texte.find('>', temp_close+1, n)
                #On supprime du texte les deux balises et ce qui se trouvait entre elles
                if temp_end < n:
                    texte = texte[:temp] + texte[temp_end+1:]
                #Si la balise fermante est tout a la fin on n'a pas de second morceau dans l'opération
                else:
                    texte = texte[:temp]
            #Sinon c'est une balise simple (ouvrante ou fermante, qu'importe)
            else :
                #On supprime la balise du texte étudié
                if end1 < n:
                    texte = texte[:temp] + texte[end1+1:]
                #Si la balise est tout a la fin on n'a pas de second morceau dans l'opération
                else:
                    texte = texte[:temp]
            
            #On met à jour la longueur du texte et le redémarrage après suppression
            n = len(texte)
            restart = temp
        #On cherche la balise suivante
        temp = texte.find('<', restart, n)
    return texte

In [11]:
#On va tester notre formatage sur le corps du premier texte
texte_test = data.iloc[0,1]
#On supprimera le contenu des balises "a" pour les liens hypertexte, ainsi que "pre" et "code" qui
#insèrent des bouts de code, et non du texte en anglais
balises_supprimer = ['a', 'pre', 'code']
#On affiche le résultat obtenu
print(formatage_texte(texte_test, balises_supprimer))

I have this image. It's a map of the UK (not including Southern Ireland):

 

I have successfully managed to get a latitude and longitude and plot it onto this map by taking the leftmost longitude and rightmost longitude of the UK and using them to work out where to put the point on the map.

This is the code (for use in Processing.js but could be used as js or anything):



However, I haven't been able to implement a Mercator projection on these values. The plots are reasonably accurate but they are not good enough and this projection would solve it.

I can't figure out how to do it. All the examples I find are explaining how to do it for the whole world.  is a good resource of examples explaining how to implement the projection but I haven't been able to get it to work.

Another resource is the  where I got the latitude and longitude values of the bounding box around the UK. They are also here:



If anyone could help me with this, I would greatly appreciate it!

Thanks



Notre fonction de formatage a l'air de marcher, nous allons donc l'appliquer à tous nos textes, avant de concaténer les titres et les corps pour n'obtenir qu'un seul bloc de texte cohérent et simple à traiter.

In [12]:
#On liste les balises dont on souhaite supprimer le contenu
balises_supprimer = ['a', 'pre', 'code']
#On applique la fonction aux titres et aux corps
Title_formated = data['Title'].apply(lambda x : formatage_texte(x, balises_supprimer))
Body_formated = data['Body'].apply(lambda x : formatage_texte(x, balises_supprimer))
#On concatène le titre et le corps formatés pour avoir un unique texte en anglais
data['Texte'] = Title_formated + '\n' + Body_formated
data.drop(columns=['Title', 'Body'], inplace=True)
data.head()

,Tags,Texte
0,<javascript><geolocation><projection><processi...,Mercator longitude and latitude calculations t...
1,<c++><file-format><autocad><cad><solid-bodies>,Starting point for learning CAD/CAE file forma...
2,<c#><.net><linq><datatable><.net-3.5>,LINQ query on a DataTable\nI'm trying to perfo...
3,<c++><performance><optimization><stl><c++-faq>,How to overload std::swap()\n is used by many ...
4,<c><file><directory><cross-platform><common-ta...,How do you get a directory listing in C?\nHow ...


On laisse les tags sous ce format pour l'export en .csv, on les transformera en listes lors de l'exploitation dans l'approche supervisée.

## Nettoyage des textes

Commençons par supprimer les nombres ainsi que les sauts de lignes et espacements multiples dans nos textes.

In [13]:
#On supprime les nombres
data['Texte'].replace(r'\d+', '', regex=True, inplace=True)
data['Texte'].replace(rf"[{re.escape(string.whitespace)}]+", ' ', regex=True, inplace=True)
print(data['Texte'].iloc[3])

How to overload std::swap() is used by many std containers (such as and ) during sorting and even assignment. But the std implementation of is very generalized and rather inefficient for custom types. Thus efficiency can be gained by overloading with a custom type specific implementation. But how can you implement it so it will be used by the std containers? 


A présent nous allons préparer un second format de texte, sans ponctuations, dans lequel les mots seront lemmatisés, triés eet mis en minuscule. Ce format servira à faire des vectorisations non contextuelles telles que la méthode "*Bag of Words*".

In [14]:
#On supprime les signes de ponctuation, en isolant le caractère ' (puisqu'utilisé pour délimiter le pattern recherché) 
data['Texte_clean'] = data['Texte'].replace(r'[.,!?:;-=..."@#_()]', '', regex=True)
data['Texte_clean'].replace(r"'", '', regex=True, inplace=True)
#On remplace les majuscules par leurs versions minuscules
data['Texte_clean'] = data['Texte_clean'].apply(lambda x : x.lower())
#On vérifie que cela a fonctionné sur un des textes
print(data['Texte_clean'].iloc[3])

how to overload stdswap is used by many std containers such as and  during sorting and even assignment but the std implementation of is very generalized and rather inefficient for custom types thus efficiency can be gained by overloading with a custom type specific implementation but how can you implement it so it will be used by the std containers 


In [15]:
data.head()

,Tags,Texte,Texte_clean
0,<javascript><geolocation><projection><processi...,Mercator longitude and latitude calculations t...,mercator longitude and latitude calculations t...
1,<c++><file-format><autocad><cad><solid-bodies>,Starting point for learning CAD/CAE file forma...,starting point for learning cad/cae file forma...
2,<c#><.net><linq><datatable><.net-3.5>,LINQ query on a DataTable I'm trying to perfor...,linq query on a datatable im trying to perform...
3,<c++><performance><optimization><stl><c++-faq>,How to overload std::swap() is used by many st...,how to overload stdswap is used by many std co...
4,<c><file><directory><cross-platform><common-ta...,How do you get a directory listing in C? How d...,how do you get a directory listing in c how do...


A présent nous allons retirer les "*stopwords*" qui sont des mots de structure et apportent peu de sens à la phrase, avant de lemmatiser les mots (i.e. les ramener à une forme canonique, l'infinitif pour les verbes, le singulier pour les noms...). Nous en profiterons pour effectuer du "*POS tagging*", c'est-à-dire trier les mots selon leur fonction grammaticale.

In [16]:
#Une fonction permettant de retirer les mots présents dans une liste de stopwords donnée en paramètres
def remove_stopwords(texte, stop):
    words = texte.split(' ')
    noise_free_words = [word for word in words if word not in stop] 
    return " ".join(noise_free_words)

#Une fonction qui utilisera le lemmatiseur donné en paramètres pour transofrmer et filtrer les mots dans nos textes
def lemmatizer_noms(texte, nlp, types_retenus=[]):  
    #On lemmatise le texte
    temp = nlp(texte)
    #Par défaut on retient tous les mots obtenus
    if types_retenus == []:
        result = [mot.lemma_ for mot in temp if len(mot.lemma_) > 1]
    #Sinon on ajoute un tri sur la nature du mot
    else :
        result = [mot.lemma_ for mot in temp if (len(mot.lemma_) > 1 and mot.pos_ in types_retenus)]
        #S'il n'y a aucun mot conservé par notre traitement, on renvoie une phrase "NONE"
        if result == []:
            result = ['NONE']
    #On renvoie la liste de mots lemmatisés
    return ' '.join(result)

In [17]:
#On utilisera les stopwords de nltk
nltk.download('stopwords')
stop = set(stopwords.words('english'))
#On utilisera le lemmatiseur de spacy
nlp = spacy.load('en_core_web_sm')
#On retiendra les noms et noms propres
types_retenus = ['NOUN', 'PROPN', 'VER']

#On applique nos fonctions précédemment codées pour transformer notre texte
data['Texte_clean'] = data['Texte_clean'].apply(lambda x: remove_stopwords(x, stop))
data['Texte_clean'] = data['Texte_clean'].apply(lambda x: lemmatizer_noms(x, nlp, types_retenus))
data.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\monte\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Tags,Texte,Texte_clean
0,<javascript><geolocation><projection><processi...,Mercator longitude and latitude calculations t...,mercator longitude latitude calculation map uk...
1,<c++><file-format><autocad><cad><solid-bodies>,Starting point for learning CAD/CAE file forma...,point cad cae file stress analysis software un...
2,<c#><.net><linq><datatable><.net-3.5>,LINQ query on a DataTable I'm trying to perfor...,linq query datatable linq query object query e...
3,<c++><performance><optimization><stl><c++-faq>,How to overload std::swap() is used by many st...,overload stdswap container assignment implemen...
4,<c><file><directory><cross-platform><common-ta...,How do you get a directory listing in C? How d...,directory scan directory folder file cross pla...


Il ne reste plus qu'à sauvegarder notre base de données : les tags sous la forme d'un string où chaque tag est entre chevrons et le texte sous deux formats explpoitables différents

In [18]:
data.to_csv(path_or_buf='database_clean.csv', index = False)